In [1]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# read from melodyData.txt
with open('generated/melodyData.txt', 'r') as f:
    parts = eval(f.read())
    print ("Number of parts:" + str(len(parts)))

# one-hot encoding

encodings = {}
encodingIndex = 0
for part in parts:
    for note in part:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

decodings = {}
for k, v in encodings.items():
    decodings[v] = k        

# encode everything in a

data_encoded = []
for part in parts:
    data_encoded.append([encodings[note] for note in part])


Number of parts:11554


In [3]:
num_data = min(15, len(data_encoded))

# X_train should contain sequences from parts 1-100
# X_test should contain the answers to said sequences
# Y_train should contains sequences from parts 201-210
# Y_test should contain the answers to said sequences

def generate_data(data_encoded, encodings):
    X = []
    Y = []
    # given data_encoded, generate training data by looping 
    for i in range(len(data_encoded[:100])):
        currentX = []
        currentY = []
        for j in range(len(data_encoded[i])-1):
            currentX.append(data_encoded[i][j])
            currentY.append(data_encoded[i][j+1])
        X.append(currentX)
        Y.append(currentY)

    X_onehot = []
    for seq in X:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
        X_onehot.append(onehot)
    X = X_onehot

    Y_onehot = []
    for seq in Y:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
            
        Y_onehot.append(onehot)
    Y = Y_onehot

    return X, Y

import math

X_train, Y_train = generate_data(data_encoded[:math.floor(num_data*0.7)], encodings)
X_test, Y_test = generate_data(data_encoded[math.floor(num_data*0.7):], encodings)

print(X_train[0].shape)
print(Y_test)

(364, 1159)
[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0.,

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(10, stateful=True, batch_input_shape=(1, None, len(encodings))),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model one time step at a time
for epoch in range(3):
    print(f'Epoch {epoch + 1}/10')
    for i, sequence in enumerate(X_train):
        model.reset_states()  # Reset states at the beginning of each sequence
        # Train on single time step
        for t in range(sequence.shape[0]): 
            x = sequence[t:t+1].reshape(1, 1, len(encodings))
            y = Y_train[i][t:t+1] 
            loss = model.train_on_batch(x, y)


Epoch 1/10
Epoch 2/10
Epoch 3/10


In [8]:
correct = 0.0
total = 0
for i in range(len(X_test)):
    # reset model
    model.reset_states()
    for j in range(len(X_test[i])):
        # feed it first note, get next note repeatedly
        note = X_test[i][j:j+1]
        # remove all tensorflow library prints
        pred = model.predict(np.array([note]), verbose=0) 
        # compare the max prediction with Y_test[i][j]
        if (np.argmax(pred) == np.argmax(Y_test[i][j])):
            correct += 1
        total += 1
        print(correct/total)
        



0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [11]:
import copy
# given a sequence, generate the next note (up to ten times)

model.reset_states()

# generate the first 10 notes after we input 1

#pred = model.predict(encodings[1])
one_hot = np.zeros((len(encodings)))
one_hot[1] = 1
print (np.array([one_hot]))
pred = model.predict(np.array([[one_hot]]))
for i in range(10):
    one_hot = np.zeros((len(encodings)))
    one_hot[np.argmax(pred)] = 1
    pred = model.predict(np.array([[one_hot]]))
    print (decodings[np.argmax(pred)])


[[0. 1. 0. ... 0. 0. 0.]]
1/1 [==============================] - 0s 16ms/step
(14, '1.33')
1/1 [==============================] - 0s 16ms/step
(10, '3.67')
1/1 [==============================] - 0s 17ms/step
(8, '2.33')
1/1 [==============================] - 0s 16ms/step
(15, '0.17')
1/1 [==============================] - 0s 16ms/step
(2, '1.75')
1/1 [==============================] - 0s 16ms/step
(0, '3.25')
1/1 [==============================] - 0s 16ms/step
(21, '8.00')
1/1 [==============================] - 0s 16ms/step
(3, '2.25')
1/1 [==============================] - 0s 16ms/step
(28, '1.50')
1/1 [==============================] - 0s 16ms/step
(9, '0.33')


In [2]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split